In [1]:
%cd ..
%ls

/Users/matusbojko/Desktop/DP/dp-project/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/matusbojko/Desktop/DP/dp-project
LICENSE           main.ipynb        project/
README.md         notebooks/        requirements.txt
data/             prepare.sh*       run/


In [2]:
from project.datamodule import BaseDataSets, RandomGenerator,TwoStreamBatchSampler,patients_to_slices
from project.utils import worker_init_fn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib import colors
from torchvision import transforms
from scipy.ndimage import zoom
from medpy import metric
import torch
import random
import numpy as np

In [3]:
random.seed(1337)
np.random.seed(1337)
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)
torch.mps.manual_seed(1337)


db_train = BaseDataSets(base_dir="./data/ACDC", split="train", num=None, transform=transforms.Compose([
    RandomGenerator([256, 256])
]))

db_val = BaseDataSets(base_dir="./data/ACDC", split="val")

total_slices = len(db_train)
labeled_slice = patients_to_slices("ACDC", 7)
labeled_idxs = list(range(0, labeled_slice))
unlabeled_idxs = list(range(labeled_slice, total_slices))

batch_sampler = TwoStreamBatchSampler(labeled_idxs, unlabeled_idxs, 24, 24-12)

trainloader = DataLoader(db_train,num_workers=4,batch_size=1, pin_memory=True, worker_init_fn=worker_init_fn)
valloader = DataLoader(db_val, batch_size=1, shuffle=False,num_workers=1)




| Labeled samples: 5.0%


In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def visualize_label(label_tensor):
    label_np = label_tensor.squeeze().cpu().numpy()

    class_names = ["Background", "RV", "MYO", "LV"]
    cmap = plt.get_cmap('nipy_spectral', 4)

    plt.figure(figsize=(6, 5))
    im = plt.imshow(label_np, cmap=cmap, vmin=0, vmax=3)
    plt.colorbar(im, ticks=[0, 1, 2, 3])

    # Add label legend
    patches = [mpatches.Patch(color=cmap(i), label=f"{i}: {name}") for i, name in enumerate(class_names)]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.title("Segmentation Label")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

In [6]:
import numpy as np
import torch

def compute_class_pixel_distribution(trainloader, num_classes):
    print("| Computing class pixel distribution...")

    class_counts = np.zeros(num_classes, dtype=np.int64)
    total_pixels = 0

    for batch in trainloader:
        label = batch["label"]  # shape: [1, H, W]
        label = label.long().squeeze(0)  # shape: [H, W]

        for c in range(num_classes):
            class_counts[c] += torch.sum(label == c).item()

        total_pixels += label.numel()

    class_freqs = class_counts / total_pixels
    print("| Class pixel counts:", class_counts)
    print("| Class frequencies:", class_freqs)

    return class_freqs


def get_alpha_from_freqs(freqs):
    inv_freqs = 1.0 / (freqs + 1e-6)  # Avoid division by zero
    alpha = inv_freqs / np.sum(inv_freqs)  # Normalize
    print("| Computed alpha (class weights):", alpha)
    return alpha.tolist()


In [7]:
num_classes = 4  # ACDC has 4 classes: background, RV, MYO, LV

freqs = compute_class_pixel_distribution(trainloader, num_classes)
alpha = get_alpha_from_freqs(freqs)

| Computing class pixel distribution...
| Class pixel counts: [82696537  1059178  1115352  1112165]
| Class frequencies: [0.96177516 0.01231843 0.01297174 0.01293467]
| Computed alpha (class weights): [0.00439447 0.34307564 0.32579818 0.32673171]


In [ ]:
| Computing class pixel distribution...
| Class pixel counts: [69406955   850234   960049   932202]
| Class frequencies: [0.96198882 0.01178435 0.0133064  0.01292043]
| Computed alpha (class weights): [0.00435986 0.35587794 0.31517398 0.32458822]